# CNN

**Reference:** https://github.com/vinayakumarr/Network-Intrusion-Detection

**Dataset:** http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import numpy as np
np.random.seed(69)  # for reproducibility

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, Flatten

from sklearn.preprocessing import Normalizer

In [2]:
DATA = '../data/'
MODELS = '../models/'
TEST = 0.3

In [3]:
traindata = pd.read_csv(DATA+'kddcup.data.corrected.csv', header=None)
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [4]:
from utils import preprocess

traindata, testdata=preprocess.preprocess(traindata)

In [5]:
class_to_idx={d:i for i,d in enumerate(traindata['Y'].unique())}
traindata['Y']=traindata['Y'].replace(class_to_idx)
testdata['Y']=testdata['Y'].replace(class_to_idx)

In [6]:
X = traindata.drop(columns=['Y'])
Y = traindata['Y']
C = testdata['Y']
T = testdata.drop(columns=['Y'])

In [7]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
testT = scaler.transform(T)

In [8]:
y_train= Y
y_test= C

In [9]:
y_train

0          0
1          0
2          0
3          0
4          0
          ..
3673818    5
3673819    5
3673820    5
3673821    5
3673822    5
Name: Y, Length: 3673823, dtype: int64

In [10]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))

In [11]:
model = Sequential()
model.add(Conv1D(8, 1, activation="tanh", input_shape=X_train.shape[1:]))
model.add(MaxPooling1D(2))
model.add(Conv1D(5,3, activation="tanh"))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation="relu"))
model.add(Dense(23, activation="softmax"))

In [12]:
# define optimizer and objective, compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 41, 8)             16        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 20, 8)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 18, 5)             125       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 9, 5)              0         
_________________________________________________________________
flatten (Flatten)            (None, 45)                0         
_________________________________________________________________
dropout (Dropout)            (None, 45)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                2

In [14]:
# train
history=model.fit(X_train, y_train, epochs=1, validation_split=0.25, shuffle=True, batch_size=1024)

2691/2691 [==============================] - 206s 76ms/step - loss: 0.1251 - accuracy: 0.9707 - val_loss: 0.0078 - val_accuracy: 0.9969


In [15]:
model.evaluate(X_test, y_test,batch_size=512)

2392/2392 [==============================] - 4s 2ms/step - loss: 0.0238 - accuracy: 0.9980


[0.023768292739987373, 0.9980450868606567]

In [16]:
model.save(MODELS+"cnn_model.hdf5")